In [144]:
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from keras import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error

In [145]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [146]:
df = pd.read_csv("./train.csv")
X_test = pd.read_csv("./test.csv")

In [147]:
X_train = df.iloc[ : , 1 : 57]

In [148]:
y_train = df.iloc[:, 57:]

In [149]:
X_test = X_test.iloc[ : , 1 : ]

In [150]:
train_df = X_train.drop(['X_04', 'X_10', 'X_11', 'X_23', 'X_39', 'X_40', 'X_47', 'X_48'], axis = 1)
X_test = X_test.drop(['X_04', 'X_10', 'X_11', 'X_23', 'X_39', 'X_40', 'X_47', 'X_48'], axis = 1)

In [151]:
def search_outlier(df) :
    quar1 = df.quantile(0.25)
    quar3 = df.quantile(0.75)
    IQR = quar3 - quar1
    condition = (df < (quar1 - 1.5 * IQR)) | (df > (quar3 + 1.5 * IQR))
    condition = condition.any(axis = 1)
    search_df = df[condition]
    idx = search_df.index
    
    return df.drop(idx, axis = 0, inplace = True), y_train.drop(idx, axis = 0, inplace = True)

In [152]:
search_outlier(train_df)

(None, None)

In [153]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11011 entries, 221 to 39591
Data columns (total 48 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X_01    11011 non-null  float64
 1   X_02    11011 non-null  float64
 2   X_03    11011 non-null  float64
 3   X_05    11011 non-null  float64
 4   X_06    11011 non-null  float64
 5   X_07    11011 non-null  float64
 6   X_08    11011 non-null  float64
 7   X_09    11011 non-null  float64
 8   X_12    11011 non-null  float64
 9   X_13    11011 non-null  float64
 10  X_14    11011 non-null  float64
 11  X_15    11011 non-null  float64
 12  X_16    11011 non-null  float64
 13  X_17    11011 non-null  float64
 14  X_18    11011 non-null  float64
 15  X_19    11011 non-null  float64
 16  X_20    11011 non-null  float64
 17  X_21    11011 non-null  float64
 18  X_22    11011 non-null  float64
 19  X_24    11011 non-null  float64
 20  X_25    11011 non-null  float64
 21  X_26    11011 non-null  float64
 

In [154]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train_df, y_train, test_size = 0.2)

In [155]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_tr)
X_tr = scaler.transform(X_tr)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [156]:
from sklearn.ensemble import RandomForestRegressor
modelRF = RandomForestRegressor()
modelRF.fit(X_tr, y_tr)

predRF = modelRF.predict(X_val)

In [157]:
rmseRF = mean_squared_error(y_val, predRF, squared = False)
nrmseRF = rmseRF/np.mean(np.abs(y_val))
scoreRF = 1.2 * np.sum(nrmseRF[:8]) + 1.0 * np.sum(nrmseRF[8:15])

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [158]:
modelDNN = Sequential()
modelDNN.add(Dense(56, activation = "relu", input_dim = 48))
modelDNN.add(Dense(56, activation = "relu"))
modelDNN.add(Dense(28, activation = "relu"))
modelDNN.add(Dense(14, activation = "linear"))

modelDNN.compile(optimizer = 'adam', loss = 'mse', metrics=['accuracy'])
modelDNN.fit(X_tr, y_tr, epochs = 50)

predDNN = modelDNN.predict(X_val)

Epoch 1/50
276/276 [==============================] - 1s 834us/step - loss: 72.4420 - accuracy: 0.9482
Epoch 2/50
276/276 [==============================] - 0s 871us/step - loss: 3.6188 - accuracy: 0.9959
Epoch 3/50
276/276 [==============================] - 0s 890us/step - loss: 2.9166 - accuracy: 0.9959
Epoch 4/50
276/276 [==============================] - 0s 858us/step - loss: 2.5657 - accuracy: 0.9959
Epoch 5/50
276/276 [==============================] - 0s 857us/step - loss: 2.3068 - accuracy: 0.9959
Epoch 6/50
276/276 [==============================] - 0s 882us/step - loss: 2.0867 - accuracy: 0.9959
Epoch 7/50
276/276 [==============================] - 0s 873us/step - loss: 1.8588 - accuracy: 0.9959
Epoch 8/50
276/276 [==============================] - 0s 884us/step - loss: 1.7264 - accuracy: 0.9959
Epoch 9/50
276/276 [==============================] - 0s 875us/step - loss: 1.6527 - accuracy: 0.9959
Epoch 10/50
276/276 [==============================] - 0s 868us/step - loss: 1.60

In [159]:
rmseDNN = mean_squared_error(y_val, predDNN, squared = False)
nrmseDNN = rmseDNN/np.mean(np.abs(y_val))
scoreDNN = 1.2 * np.sum(nrmseDNN[:8]) + 1.0 * np.sum(nrmseDNN[8:15])

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [160]:
from xgboost import XGBRegressor
modelXGB = XGBRegressor()
modelXGB.fit(X_tr, y_tr)

predXGB = modelXGB.predict(X_val)

In [161]:
rmseXGB = mean_squared_error(y_val, predXGB, squared = False)
nrmseXGB = rmseXGB/np.mean(np.abs(y_val))
scoreXGB = 1.2 * np.sum(nrmseXGB[:8]) + 1.0 * np.sum(nrmseXGB[8:15])

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [162]:
print(scoreRF, scoreDNN, scoreXGB)

3.8190405973098676 3.9561385259427713 3.9941778828498973
